In [2]:
import sqlite3
import pandas as pd
import numpy as np
import sys
sys.path.append('C:/Users/Archiebald/Moons/Task_1')
from jupiter import Moons

In [3]:
sys.path.append('C:/Users/Archiebald/Moons/Task_1')

In [4]:
jupiter_moons = Moons("jupiter_kepler.db", "data/")

In [5]:
jupiter_moons.all_data().head()

,moon,period_days,distance_km,radius_km,mag,mass_kg,group,ecc,inclination_deg
0,Adrastea,0.298,129000.0,8.2,NaN,NaN,Inner,0.001500,0.03
1,Aitne,730.000,23231000.0,1.5,22.7,NaN,Carme,0.264000,165.00
2,Amalthea,0.498,181400.0,83.5,14.1,2.080000e+18,Inner,0.003190,0.37
3,Ananke,630.000,21300000.0,14.0,18.9,NaN,Ananke,0.174725,148.60
4,Aoede,761.000,24000000.0,2.0,22.5,NaN,Pasiphae,0.432000,158.00


In [7]:
jupiter_moons.convert_distance("km", "radius_km")
jupiter_moons.convert_time("days","period_days")
jupiter_moons.gen_sma("semi_major_axis","radius_m","ecc")
jupiter_moons.all_data().head()

,moon,period_days,distance_km,radius_km,mag,mass_kg,group,ecc,inclination_deg,radius_m,period_s,semi_major_axis
0,Adrastea,0.298,129000.0,8.2,NaN,NaN,Inner,0.001500,0.03,8200.0,25747.2,8212.318478
1,Aitne,730.000,23231000.0,1.5,22.7,NaN,Carme,0.264000,165.00,1500.0,63072000.0,2038.043478
2,Amalthea,0.498,181400.0,83.5,14.1,2.080000e+18,Inner,0.003190,0.37,83500.0,43027.2,83767.217424
3,Ananke,630.000,21300000.0,14.0,18.9,NaN,Ananke,0.174725,148.60,14000.0,54432000.0,16964.038178
4,Aoede,761.000,24000000.0,2.0,22.5,NaN,Pasiphae,0.432000,158.00,2000.0,65750400.0,3521.126761
